# using `qcodes`/`qchar` with `qchart`

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import qcodes as qc
from pathlib import Path
import numpy as np

pyqtgraph plotting not supported, try "from qcodes.plots.pyqtgraph import QtPlot" to see the full error


In [2]:
# first some imports to handle the data saving
from qcodes import initialise_database
from qcodes.dataset.experiment_container import load_or_create_experiment

db_path = Path('../_logs/data/test_plots.db')
db_path.parents[0].mkdir(parents=True, exist_ok=True)
qc.config.current_config.core.db_location = str(db_path)

initialise_database()  # just in case no database file exists
load_or_create_experiment("qchart", sample_name="None")

qchart#None#2@C:\Users\Nik\Dropbox\Repos\qchart\_logs\data\test_plots.db
------------------------------------------------------------------------
36-results-1-zed_ch1,zed_ch2,bob_v1,bob_v2-0
37-results-2-zed_ch1,zed_ch2,bob_v1,bob_v2-0
38-results-3-zed_ch1,zed_ch2,bob_v1,bob_v2-300
39-results-4-zed_ch1,zed_ch2,bob_v1,bob_v2-300
40-results-5-zed_ch1,zed_ch2,bob_v1,bob_v2-316
41-results-6-zed_ch1,zed_ch2,bob_v1,bob_v2-500
42-results-7-zed_ch1,zed_ch2,bob_v1,bob_v2-500
43-results-8-zed_ch1,zed_ch2,bob_v1,bob_v2-500
44-results-9-zed_ch1,zed_ch2,bob_v1,bob_v2-304
45-results-10-zed_ch1,zed_ch2,bob_v1,bob_v2-282
46-results-11-zed_ch1,zed_ch2,bob_v1,bob_v2-500
47-results-12-zed_ch1,zed_ch2,bob_v1,bob_v2-500
48-results-13-zed_ch1,zed_ch2,bob_v1,bob_v2-500
49-results-14-zed_ch1,zed_ch2,bob_v1,bob_v2-500
50-results-15-zed_ch1,zed_ch2,bob_v1,bob_v2-500
51-results-16-zed_ch1,zed_ch2,bob_v1,bob_v2-500
52-results-17-zed_ch1,zed_ch2,bob_v1,bob_v2-500
53-results-18-zed_ch1,zed_ch2,bob_v1,bob_v2-300

### Set up subscriber as default

And change update frequency to be a little faster than the qcodes defaults

In [3]:
qc.config.current_config.subscription.subscribers['qchart'] = {
    'factory': 'qchart.subscribers.QcodesSubscriber',
    'factory_kwargs': {
        'subscriber_logs': False
    },
    'subscription_kwargs': {
        'min_wait': 0,
        'min_count': 1,
        'callback_kwargs': {}
    }
}

qc.config.current_config.subscription.default_subscribers = ['qchart']

### Start listener

In [15]:
from qchart import start_listener, listener_is_running

start_listener()

starting qchart listener at c:\users\nik\dropbox\repos\qchart\listener_start.py ...
qchart listener successfully started.


In [8]:
# check
listener_is_running()

True

### Setup DummyInstr

In [7]:
# and the imports I need to do the measurement
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_dataset

# making some fake instruments
try:
    dac = DummyInstrument('zed', gates=['ch1', 'ch2']) # to set
    dmm = DummyInstrument('bob', gates=['v1', 'v2'])   # to get
except KeyError:
    pass

In [8]:
from functools import partial

# and we'll make a 2D gaussian to sample from/measure
def gauss_model(x0: float, y0: float, sigma: float, noise: float=0.0005):
    """
    Returns a generator sampling a gaussian. The gaussian is
    normalised such that its maximal value is simply 1
    """
    while True:
        (x, y) = yield
        model = np.exp(-((x0-x)**2+(y0-y)**2)/2/sigma**2)*np.exp(2*sigma**2)
        noise = np.random.randn()*noise
        yield (model + noise)
        
# connect the dmm.v1 to "measure" the gaussian

gauss1 = gauss_model(0.1, 0.2, 0.25)
next(gauss1)

gauss2 = gauss_model(-0.2, 0.1, 0.5)
next(gauss2)

def measure_gauss1(dac):
    val = gauss1.send((dac.ch1.get(), dac.ch2.get()))
    next(gauss1)
    return val

def measure_gauss2(dac):
    val = gauss2.send((dac.ch1.get(), dac.ch2.get()))
    next(gauss2)
    return val*1e-9

dmm.v1.get = partial(measure_gauss1, dac)
dmm.v2.get = partial(measure_gauss2, dac)

### sweeps with a 'grid'

In [9]:
from qchart.subscribers import QcodesSubscriber

def do1d(param_set, xarray, delay, *param_meas):

    meas = Measurement()
    meas.write_period = 0.5
    
    meas.register_parameter(param_set)
    param_set.post_delay = 0
    
    output = []
    for pm in param_meas:
        meas.register_parameter(pm, setpoints=(param_set,))
        output.append([pm, None])
        
    with meas.run() as ds:

        for x in xarray:
            param_set.set(x)
            time.sleep(delay)
            for i, parameter in enumerate(param_meas):
                output[i][1] = parameter.get()
            
            ds.add_result((param_set, x),
                                 *output)
    
        return ds.dataset

def do2d_grid(param_set1, xarray, delay1,
              param_set2, yarray, delay2,
              *param_meas):

    meas = Measurement()
    meas.write_period = 0.5
    
    meas.register_parameter(param_set1)
    param_set1.post_delay = 0
    meas.register_parameter(param_set2)
    param_set2.post_delay = 0

    output = []
    for parameter in param_meas:
        meas.register_parameter(parameter, setpoints=(param_set1,param_set2))
        output.append([parameter, None])

    with meas.run() as ds:
        
        for y in yarray:
            param_set2.set(y)
            time.sleep(delay2)
            for x in xarray:
                param_set1.set(x)
                time.sleep(delay1)
                for i, parameter in enumerate(param_meas):
                    output[i][1] = parameter.get()
                ds.add_result((param_set1, x),
                                     (param_set2, y),
                                     *output)
        time.sleep(0.2)
        
        return ds.dataset  

In [34]:
xarr = np.linspace(-1,2,150)
ds = do1d(dac.ch1, xarr, 0.2, dmm.v1, dmm.v2)

Starting experimental run with id: 73


In [32]:
xarr = np.linspace(-1,2,15)
yarr = np.linspace(-1,1,10)
ds = do2d_grid(dac.ch1, xarr, 0.2, dac.ch2, yarr, 0.3, dmm.v1, dmm.v2)

Starting experimental run with id: 71


### sweeps with ridiculous spacing

In [19]:
def get_x():
    return np.random.uniform(low=-1.0, high=2.0)

def get_y():
    return np.random.uniform(low=-1.0, high=1.0)

def do2d_rand(param_set1, param_set2, n, delay,
              *param_meas):

    meas = Measurement()
    meas.write_period = 0.5
    
    meas.register_parameter(param_set1)
    param_set1.post_delay = 0
    meas.register_parameter(param_set2)
    param_set2.post_delay = 0

    output = []
    for parameter in param_meas:
        meas.register_parameter(parameter, setpoints=(param_set1,param_set2))
        output.append([parameter, None])

    with meas.run() as ds:
        
        for i in range(n):
                
                x = get_x()
                param_set1.set(x)
                
                y = get_y()
                param_set2.set(y)
                
                time.sleep(delay)
                for i, parameter in enumerate(param_meas):
                    output[i][1] = parameter.get()
                ds.add_result((param_set1, x),
                                     (param_set2, y),
                                     *output)
        time.sleep(0.2)
        
    return ds.run_id  # convenient to have for plotting

In [27]:
ds = do2d_rand(dac.ch1, dac.ch2, 250, 0.1, dmm.v1, dmm.v2)

Starting experimental run with id: 66
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!


6